# Fourier-Domain OCT

Fourier-Domain OCT revolutionized the field by eliminating mechanical scanning and dramatically increasing speed.

## Learning Objectives

By the end of this notebook, you will understand:
1. The fundamental principle of FD-OCT
2. How spectral information encodes depth
3. Spectral-Domain OCT (SD-OCT)
4. Swept-Source OCT (SS-OCT)
5. Advantages over TD-OCT

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import fft, fftfreq, fftshift

plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

print("Libraries loaded successfully!")

## 1. The Key Insight

In FD-OCT, instead of scanning the reference mirror, we:
1. Capture the **entire interference spectrum** simultaneously
2. Apply **Fourier transform** to extract depth information
3. Get all depths at once - no mechanical scanning!

### The Fourier Relationship

The interference spectrum $I(k)$ and the depth profile $A(z)$ are Fourier pairs:

$$A(z) = \mathcal{F}^{-1}[I(k)]$$

where $k = 2\pi/\lambda$ is the wavenumber.

In [ ]:
def generate_fd_oct_spectrum(wavelengths, sample_structure):
    """
    Generate FD-OCT interference spectrum.
    
    Parameters:
    - wavelengths: Array of wavelengths (m)
    - sample_structure: List of (depth, reflectivity) tuples
    """
    # Wavenumbers
    k = 2 * np.pi / wavelengths
    
    # Start with source spectrum (Gaussian)
    lambda0 = np.mean(wavelengths)
    sigma = (wavelengths[-1] - wavelengths[0]) / 6  # ~99% of spectrum
    source_spectrum = np.exp(-((wavelengths - lambda0) / sigma)**2)
    
    # Initialize interference spectrum
    spectrum = source_spectrum.copy()
    
    # Add interference modulation from each reflector
    for depth, reflectivity in sample_structure:
        # Modulation frequency depends on depth
        # Higher frequency = deeper structure
        modulation = reflectivity * np.cos(2 * k * depth)
        spectrum = spectrum * (1 + modulation)
    
    return spectrum

# Define sample
sample = [
    (50e-6, 0.3),
    (150e-6, 0.4),
    (250e-6, 0.2),
]

# Generate wavelength array (typical SD-OCT)
lambda0 = 840e-9  # Center wavelength
bandwidth = 50e-9  # 50 nm bandwidth
wavelengths = np.linspace(lambda0 - bandwidth/2, lambda0 + bandwidth/2, 2048)

# Generate spectrum
spectrum = generate_fd_oct_spectrum(wavelengths, sample)

# Plot
plt.figure(figsize=(14, 6))
plt.plot(wavelengths * 1e9, spectrum, 'b', linewidth=2)
plt.xlabel('Wavelength (nm)', fontsize=12)
plt.ylabel('Intensity (a.u.)', fontsize=12)
plt.title('FD-OCT Interference Spectrum', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("Notice the spectral modulation - this encodes depth information!")
print("Faster oscillations = deeper structures")

## 2. Fourier Transform to Extract A-Scan

Apply inverse Fourier transform to recover the depth profile.

In [ ]:
# Resample to linear k-space (required for FFT)
k = 2 * np.pi / wavelengths
k_linear = np.linspace(k.min(), k.max(), len(k))
spectrum_k = np.interp(k_linear, k[::-1], spectrum[::-1])

# Apply Fourier transform
a_scan_complex = fft(spectrum_k)
a_scan = np.abs(a_scan_complex)

# Generate depth axis
# Depth range determined by spectral sampling
dk = k_linear[1] - k_linear[0]
depth_max = np.pi / dk
depths = np.linspace(0, depth_max, len(a_scan))

# Convert to dB
a_scan_db = 20 * np.log10(a_scan / np.max(a_scan))
a_scan_db = np.clip(a_scan_db, -60, 0)

# Plot
plt.figure(figsize=(14, 6))
plt.plot(depths[:len(depths)//2] * 1e6, a_scan_db[:len(depths)//2], 'g', linewidth=2)
plt.xlabel('Depth (μm)', fontsize=12)
plt.ylabel('Reflectivity (dB)', fontsize=12)
plt.title('Reconstructed A-Scan (from Fourier Transform)', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)

# Mark expected layer positions
for depth, refl in sample:
    plt.axvline(depth * 1e6, color='r', linestyle='--', alpha=0.5, label=f'{depth*1e6:.0f} μm')

plt.xlim(0, 400)
plt.legend()
plt.tight_layout()
plt.show()

print("The A-scan is recovered from a single spectral measurement!")
print("No mechanical scanning required.")

## 3. Spectral-Domain OCT (SD-OCT)

SD-OCT uses:
- **Broadband light source** (SLD, Ti:Sapphire laser)
- **Spectrometer** with line camera to capture spectrum

### Advantages:
- All wavelengths detected simultaneously
- Very fast (20,000-100,000 A-scans/sec)
- High sensitivity

### Limitations:
- Spectrometer resolution limits imaging depth
- Sensitivity decreases with depth

## 4. Swept-Source OCT (SS-OCT)

SS-OCT uses:
- **Tunable laser** that rapidly sweeps through wavelengths
- **Photodetector** (not a spectrometer)

### Advantages:
- Can use longer wavelengths (1050 nm, 1300 nm)
- Better tissue penetration
- More uniform sensitivity vs depth
- Very high speed possible (400,000+ A-scans/sec)

### Applications:
- Anterior segment imaging
- Cardiovascular imaging
- Dermatology

In [ ]:
def simulate_ss_oct(sample_structure, sweep_range=(1250e-9, 1350e-9), num_points=2048):
    """
    Simulate Swept-Source OCT acquisition.
    """
    # Wavelength sweep
    wavelengths = np.linspace(sweep_range[0], sweep_range[1], num_points)
    k = 2 * np.pi / wavelengths
    
    # Time points (assuming linear sweep)
    time = np.linspace(0, 1, num_points)  # Normalized time
    
    # Detector signal vs time
    signal = np.ones(num_points)
    
    for depth, reflectivity in sample_structure:
        # Each depth creates a beat frequency in the detector signal
        beat = reflectivity * np.cos(2 * k * depth)
        signal += beat
    
    return time, wavelengths, signal

# Simulate SS-OCT
sample_ss = [(100e-6, 0.4), (200e-6, 0.3), (300e-6, 0.2)]
time, wavelengths_ss, signal_ss = simulate_ss_oct(sample_ss)

# Plot
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))

# Signal vs time
ax1.plot(time, signal_ss, 'b', linewidth=2)
ax1.set_xlabel('Time (normalized)', fontsize=12)
ax1.set_ylabel('Detector Signal', fontsize=12)
ax1.set_title('SS-OCT: Detector Signal During Wavelength Sweep', fontsize=13, fontweight='bold')
ax1.grid(True, alpha=0.3)

# Signal vs wavelength
ax2.plot(wavelengths_ss * 1e9, signal_ss, 'r', linewidth=2)
ax2.set_xlabel('Wavelength (nm)', fontsize=12)
ax2.set_ylabel('Detector Signal', fontsize=12)
ax2.set_title('SS-OCT: Interference Spectrum', fontsize=13, fontweight='bold')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("SS-OCT: Laser sweeps wavelength, detector records beat frequencies")
print("Beat frequencies encode depth information")

## 5. Speed and Sensitivity Advantages

FD-OCT has a fundamental sensitivity advantage over TD-OCT.

In [ ]:
# Comparison of OCT technologies
comparison = {
    'Property': ['A-scan Rate\n(per second)', 'Sensitivity\n(dB)', 
                 'Imaging Depth\n(mm)', 'Common\nWavelengths (nm)'],
    'TD-OCT': ['100-400', '80-100', '1-2', '840'],
    'SD-OCT': ['20,000-100,000', '95-110', '2-3', '840, 1050'],
    'SS-OCT': ['100,000-400,000', '100-115', '3-5', '1050, 1300']
}

import pandas as pd
df = pd.DataFrame(comparison)

print("="*80)
print("OCT Technology Comparison")
print("="*80)
print(df.to_string(index=False))
print("="*80)

# Visualize speed comparison
fig, ax = plt.subplots(figsize=(10, 6))

technologies = ['TD-OCT', 'SD-OCT', 'SS-OCT']
speeds = [250, 50000, 200000]  # Representative values (A-scans/sec)
colors = ['red', 'orange', 'green']

bars = ax.bar(technologies, speeds, color=colors, alpha=0.7, edgecolor='black', linewidth=2)

ax.set_ylabel('A-scan Rate (per second)', fontsize=12)
ax.set_title('Speed Comparison of OCT Technologies', fontsize=14, fontweight='bold')
ax.set_yscale('log')
ax.grid(True, axis='y', alpha=0.3)

# Add value labels
for bar, speed in zip(bars, speeds):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2, height,
            f'{speed:,}\nA-scans/s',
            ha='center', va='bottom', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.show()

print("\nKey Advantages of FD-OCT:")
print("  ✓ 50-1000x faster than TD-OCT")
print("  ✓ 15-20 dB better sensitivity")
print("  ✓ Enables 3D volumetric imaging")
print("  ✓ Reduces motion artifacts")
print("  ✓ Enables real-time imaging")

## 6. Why FD-OCT is More Sensitive

### Signal-to-Noise Ratio (SNR)

TD-OCT: Each depth point measured separately
- SNR limited by detector noise at each measurement

FD-OCT: All depths measured simultaneously
- **Multiplex advantage**: SNR improves with number of spectral points
- Sensitivity gain = $\sqrt{N}$ where N is number of pixels

For N = 1024 pixels: **32 dB sensitivity improvement!**

## Summary

In this notebook, we've learned:

1. ✅ **FD-OCT principle**: Spectral information → Fourier transform → Depth profile

2. ✅ **No mechanical scanning**: All depths acquired simultaneously

3. ✅ **SD-OCT**: Uses spectrometer, excellent for retinal imaging

4. ✅ **SS-OCT**: Uses tunable laser, best for deep tissue imaging

5. ✅ **Major advantages**: 50-1000x faster, 15-20 dB better sensitivity

## Key Equations

**Fourier relationship:**
$$A(z) = \mathcal{F}^{-1}[I(k)]$$

**Imaging depth (SD-OCT):**
$$z_{max} = \frac{\lambda_0^2}{4n\delta\lambda}$$

**Sensitivity advantage:**
$$\text{SNR gain} = 10\log_{10}(N) \text{ dB}$$

## Technology Selection

**Choose SD-OCT for:**
- Retinal imaging
- High speed at moderate depth
- 840 nm wavelength

**Choose SS-OCT for:**
- Deep tissue penetration
- Anterior segment
- 1050 nm or 1300 nm wavelength
- Maximum speed

## Next Steps

In the next notebook, we'll explore:
- Signal processing techniques
- Dispersion compensation
- Noise reduction
- Image enhancement

---

**Continue to Notebook 6: Signal Processing →**